In [59]:
import json
import tensorflow as tf
import json
%matplotlib notebook
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
import os

In [2]:
data_path = './event_sequence_data_20200607_hour_08.json'

with open(data_path, 'r') as f:
    raw_data = json.load(f)

In [4]:
raw_data[0]
raw_data_small = raw_data[:50000]
neg_data = []
pos_data = []

for r in raw_data_small:
    if r['fraud_prob'] > 0:
        pos_data.append(r)
    else:
        neg_data.append(r)

In [70]:
times = []
event_types = []
labels = []

vocab = ['na', 'start', 'view', 'click', 'install']
str2idx = {u:i for i, u in enumerate(vocab)}
idx2str= np.array(vocab)

for row in raw_data_small:
    _event_type_list, _time_list = list(zip(*row['sequence']))
    _event_type_list = tuple(str2idx[c] for c in _event_type_list)
    # drop missing times for data points 5096332 and 8959733
    # otherwise cause problems in creating embeddings
    if not any(_time_list): 
        continue
        
    labels.append(row['fraud_prob'])    
    event_types.append(_event_type_list)

    times.append(_time_list)    

In [28]:
padding_len = 20 
padding_type = 'post'
truncating_type = 'pre'

padded_event_types = tf.keras.preprocessing.sequence.pad_sequences(
    event_types, 
    padding=padding_type, 
    truncating = truncating_type,
    maxlen=padding_len,
    value=0,
    dtype=object,
)
padded_event_types = padded_event_types.astype(int)

array([[1, 2, 3, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       ...,
       [1, 2, 0, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=object)

In [67]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = tf.data.Dataset.from_tensor_slices(padded_event_types)
dataset = dataset.map(split_input_target)

BATCH_SIZE = 10
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

for input_example, target_example in  dataset.take(1):
    print('input_example:{}'.format(input_example))
    print('target_example:{}'.format(target_example))


input_example:[[1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 3 2 2 1 2 3 0 0 0 0 0 0 0 0 0 0]]
target_example:[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 3 2 2 1 2 3 0 0 0 0 0 0 0 0 0 0 0]]


In [63]:
vocab_size = len(vocab)
embedding_dim = 16
rnn_units = 32

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Train for 500 steps
Epoch 1/5
500/500 [==============================] - 7s 14ms/step - loss: 0.3405
Epoch 2/5
500/500 [==============================] - 5s 11ms/step - loss: 0.2035
Epoch 3/5
500/500 [==============================] - 5s 11ms/step - loss: 0.2001
Epoch 4/5
500/500 [==============================] - 5s 11ms/step - loss: 0.1956
Epoch 5/5
500/500 [==============================] - 5s 11ms/step - loss: 0.1912


In [69]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 16)             80        
_________________________________________________________________
gru_7 (GRU)                  (1, None, 32)             4800      
_________________________________________________________________
dense_7 (Dense)              (1, None, 5)              165       
Total params: 5,045
Trainable params: 5,045
Non-trainable params: 0
_________________________________________________________________


In [111]:
def generate_event(model):
    length = 20
    input_eval = [1]
    input_eval = tf.expand_dims(input_eval, 0)

    full_seq_str = []
    full_seq_int = []

    # Here batch size == 1
    model.reset_states()
    for i in range(length):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # sample an action
        predictions = predictions 
        predicted_idx = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # pass the predicted action as the next input to the model
        input_eval = tf.expand_dims([predicted_idx], 0)

        full_seq_str.append(idx2str[predicted_idx] + " ")
        full_seq_int.append(predicted_idx)

    return 'start ' + ''.join(full_seq_str), full_seq_int


In [112]:

for i in range(20):
    seq_str, seq_int = generate_event(model)
    print(seq_str)


start view view view view click view view start view install install view click view start install start install click view 
start start click view install start view click view view view install click start view view install install view view na 
start click install install click install na click na na na na na na na na na na na na na 
start click click view view view view install start na install click view view start start start install start click view 
start view start view start click click click install click view install start view install start click start start view install 
start start start click view view view na na na na na na na na na na na na na na 
start view view start start view view view install click start start start click view install click view click click view 
start view view click install start view view click start install click view install start click click view start view install 
start view install view click view install start view click install click v

In [113]:
seq_int

[2, 1, 3, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 1, 2, 2, 2, 3, 4, 4]

In [127]:
# Discriminator - CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(len(vocab), 16, input_length=20))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(MaxPooling1D(3))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1))
cnn_model.summary()
cnn_model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy',metrics=['acc'])
# history = model.fit(x_train, y_train,
# epochs=10, batch_size=128, validation_split=0.2)
cnn_model.predict(np.array([seq_int]))

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 20, 16)            80        
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 18, 32)            1568      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 6, 32)             0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 4, 32)             3104      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 4,785
Trainable params: 4,785
Non-trainable params: 0
___________________________________________________

array([[0.02621368]], dtype=float32)

In [97]:
class Environment(object):
    def __init__(self, discriminator, rollout_policy, path):
        self.path = path
        self.discriminator = discriminator
        self.rollout_policy = rollout_policy # an pretrained RNN model
        self.base_state = [1]
        self.state = self.base_state
        self.reset()
        self.T = 20
        self.n_sample = 100
    
    def load_model():
        tf.train.latest_checkpoint(checkpoint_dir)
        model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
        model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
        model.build(tf.TensorShape([1, None]))
        model.summary()
        
    def get_state(self):
        return self.state
    
    def reset(self):
        self.t = t
        self.rollout_policy.reset_states()
        self.state = self.base_state
        return True
    
    def step(self, action):
        self.t = self.t + 1

        reward = self.get_reward(action, self.n_sample)
        is_episode_end = self.t > self.T

        self.state.append(action)
        next_state = self.get_state()

        return [next_state, reward, is_episode_end]


    def get_reward(self, action, n_sample):
        reward = 0
        for idx_sample in range(n_sample):
            full_seq = self.generate_event(state, self.T)
            reward += self.discriminator.predict(full_seq) / n_sample
        return reward
        
    def generate_event(state, length):
        input_eval = state
        input_eval = tf.expand_dims(input_eval, 0)
        full_seq = input_eval
        # Empty string to store our results
        text_generated = []

        self.rollout_policy.reset_states()
        for i in range(length):
            predictions = rollout_policy(input_eval)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions
            predicted_idx = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            
            # pass the predicted action as the next input to the model
            input_eval = tf.expand_dims([predicted_idx], 0)
            text_generated.append(idx2str[predicted_idx])
            full_seq.append(predicted_idx)
        return full_seq

    

In [ ]:
# plicy gradient update
log_prob = tf.log(tf.reduce_mean(prob * action, axis=-1))
loss = - log_prob * reward
optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
minimize = optimizer.minimize(loss)
